In [1]:
from make_predictions import *

# FILE_PATH = '/home/sam/Hhd/twitter_sentiment/'
FILE_PATH = '/home/sam/Data/twitter_sentiment/'

Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 965M (0000:01:00.0)


### AirCanada

In [2]:
AC = GenerateData(FILE_PATH, 'AirCanada.csv', max_len_train=19)
AC.prepare_data(wv_size=600)
AC.get_result(n_preview=10, n_top = 20, name='AC_result',verbose=False)

Loading csv: AirCanada.csv ...
Note: pre-process changes the dataframe inplace.
Loading existing model tweets600.model.bin ...
Done building.
max sentence length is:  17
npy already exists, loading ...
Done loading npy file.
npy already exists.
>>>Done preparing data.<<<

prediction
Done converting numeric to categorical, this changes df.
===Positive===
[('great', 7), ('inflate', 6), ('thanks', 3), ('balloon', 3), ('happy', 3), ('foundation', 3), ('support', 3), ('crepes', 2), ('airline', 2), ('ready', 2), ('jetxd', 2), ('eduaubeedubueeduaubcedubduaeduaubdedubudu', 2), ('visit', 2), ('lounge', 2), ('freshly', 2), ('flight', 2), ('search', 2), ('yyz', 2), ('made', 2), ('greatest', 2)]
===Neutral===
[('away', 5), ('looking', 5), ('big', 4), ('nwtsecrets', 4), ('heres', 4), ('w', 4), ('one', 4), ('https', 4), ('spectacularnwt', 4), ('trips', 4)]
===Negative===
[('flight', 48), ('time', 15), ('travel', 14), ('found', 12), ('trip', 12), ('hours', 11), ('didnt', 11), ('service', 11), ('let',

In [3]:
AC.check(word='bag', sentiment=3, n_view=10)

RT @ottcaplax: 2 out of 3 ain't bad. @AirCanada only managed to lose 1 bag on the way to Orlando. I mean a lacrosse player doesn't need sti…
@AirCanada I have checked and it says 1st bag free for economy on tango, does this mean hand luggage or any other baggage allowance ?
RT @ottcaplax: 2 out of 3 ain't bad. @AirCanada only managed to lose 1 bag on the way to Orlando. I mean a lacrosse player doesn't need sti…
2 out of 3 ain't bad. @AirCanada only managed to lose 1 bag on the way to Orlando. I mean a lacrosse player doesn't need sticks does he?
@AirCanada my case ?I have been hung up on, called terrible names, lied too and completely dismissed .My case is now deeper then missing bag
@British_Airways @AirCanada thank you for working together to find my bag, I hope to have it home soon?
Hey @British_Airways you lost my bag on a change from Heathrow to Toronto; you were supposed to transfer it to @AirCanada but didn't.
Sweet. Arrived in Ireland on a trip of a lifetime and @AirCanada los